# DIL THEK  HAI  AAPKA?

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("heart.csv")

In [ ]:
df.head()

In [ ]:
df.info()

age - age in years

sex - (1 = male; 0 = female)

cp - chest pain type 
     0: Typical angina: chest pain related decrease blood supply to the heart 
     1: Atypical angina: chest pain not related to heart 
     2: Non-anginal pain: typically esophageal spasms (non heart related)
     3: Asymptomatic: chest pain not showing signs of disease

trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern

chol - serum cholestoral in mg/dl serum = LDL + HDL + .2 * triglycerides 
above 200 is cause for concern

fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) '>126' mg/dL signals diabetes

restecg - resting electrocardiographic results 0: Nothing to note 1: ST-T Wave abnormality can range from mild symptoms to severe problems signals non-normal heart beat 2: Possible or definite left ventricular hypertrophy Enlarged heart's main pumping chamber

thalach - maximum heart rate achieved

exang - exercise induced angina (1 = yes; 0 = no)

oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more

slope - the slope of the peak exercise ST segment 0: Upsloping: better heart rate with excercise (uncommon) 1: Flatsloping: minimal change (typical healthy heart) 2: Downslopins: signs of unhealthy heart

ca - number of major vessels (0-3) colored by flourosopy colored vessel means the doctor can see the blood passing through the more blood movement the better (no clots)

thal - thalium stress result 1,3: normal 6: fixed defect: used to be defect but ok now 7: reversable defect: no proper blood movement when excercising

target - have disease or not (1=yes, 0=no) (= the predicted attribute)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
#import pandas_profiling 

#pandas_profiling.ProfileReport(df)

In [ ]:
df2=df.drop(['fbs','restecg'],axis=1)

In [ ]:
df2.head()

In [ ]:
X = df2.drop(['target'],axis=1)
y = df2['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,random_state=42)
rfc.fit(X_train,y_train)
print("It's show time of the Accuracy: ",rfc.score(X_test,y_test))

In [ ]:
print(rfc.predict([[69,1,0,140,254,146,0,2.0,1,3,3]]))

In [ ]:
def plot_feature_importance(rfc):
    plt.figure(figsize=(8,6))
    n_fetures=11
    plt.barh(range(n_fetures),rfc.feature_importances_,align='center')
    plt.yticks(np.arange(n_fetures),X)
    plt.xlabel("Feature Importance")
    plt.ylabel("Features")
    plt.ylim(-1,n_fetures)
    
plot_feature_importance(rfc) 

In [ ]:
print(X_test)

In [ ]:
print(y_test)

In [ ]:
#inputt=[int(x) for x in "45 32 60".split(' ')]
#final=[np.array(inputt)]
#print(final)
#b = rfc.predict_proba(final)
#print(b)
import pickle
pickle.dump(rfc,open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))

In [ ]:
from flask import Flask,request, url_for, redirect, render_template
import pickle
import numpy as np

app = Flask(__name__)
model=pickle.load(open('model.pkl','rb'))

@app.route('/')
def hello_world():
    return render_template("trytwo.html")

@app.route('/predict',methods=['POST','GET'])
def predict():
    # receive the values send by user in three text boxes thru request object -> requesst.form.values()
    int_features=[float(x) for x in request.form.values()]
    final=[np.array(int_features)]
    print(int_features)
    print(final)
    prediction=model.predict_proba(final)
    output='{0:.{1}f}'.format(prediction[0][1], 2)

    if output>str(0.5):
        return render_template('trytwo.html',pred='Your have heart disease.Take good care of yourself.your probability of having heart disease is {}'.format(output))
    else:
        return render_template('trytwo.html',pred='Your do not have heart disease.However,you need to take care of yourself.Your probability of having heart disease is {}'.format(output))

if __name__ == '__main__':
      app.run(debug=False)

import numpy as np
import pandas as pd
from flask import Flask, request, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl','rb'))


@app.route('/')
def home():
    return render_template('utsav.html')

@app.route('/predict',methods=['POST','GET'])
def predict():
    input_features = [float(x) for x in request.form.values()]
    features_value = [np.array(input_features)]
    
    features_name =  ["age","sex", "cp", "trestbps", "chol", "thalach", "exang",
                    "oldpeak", "slope", "ca", "thal"]
    df2 = pd.DataFrame(features_value, columns=features_name)
    output = model.predict(df2)
    
    if output == 1:
        res_val = "**Heart Disease**"
    else:
        res_val = "No Heart Disease"
        
    return render_template('utsav.html', prediction_text='Patient has {}'.format(res_val))
if __name__ == "__main__":
    app.run()